In [ ]:
# HW 3 - NYT Articles Notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = pd.read_csv("nyt.csv")
print(df.head())
print(df.shape)
# Shuffle and Split 8/1/1
train_df, temp_df = train_test_split(df, test_size = 0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, random_state= 42, test_size=0.5)
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

                                                text   label
0  (reuters) - carlos tevez sealed his move to ju...  sports
1  if professional pride and strong defiance can ...  sports
2  palermo, sicily — roberta vinci beat top-seede...  sports
3  spain's big two soccer teams face a pair of it...  sports
4  the argentine soccer club san lorenzo complete...  sports
(11519, 2)
(9215, 2)
(1152, 2)
(1152, 2)


Part 1: Bag Of Words (20 points):

Train a text classifier using the following document representation techniques and report accuracy,
macro-f1 score, and micro-f1 score on the test set. macro-f1 score refers to evaluating multi-class
classification models by calculating the F1 score for each class independently and then taking the
unweighted average of those scores.

a. Each document is represented as a binary-valued vector of dimension equal to the size of the
vocabulary. The value at an index is 1 if the word corresponding to that index is present in the
document, else 0.

b. A document is represented by a vector of dimension equal to the size of the vocabulary where
the value corresponding to each word is its frequency in the document.

c. Each document is represented by a vector of dimension equal to the size of the vocabulary
where the value corresponding to each word is its tf-idf value.

(a) binary-valued vector

(b) frequency vector

(c) tf-idf vector

In [ ]:
# Data Manipulation
X_train = train_df['text']
y_train = train_df['label']
X_val = val_df['text']
y_val = val_df['label']
X_test = test_df['text']
y_test = test_df['label']

In [ ]:
# (a)binary-valued vector
# TODO
# vectorizer for binary-valued
# Set binary = true, when words is true =1, else = 0
vectorizer = CountVectorizer(binary=True)
# training set fit, transform training, validation, test
X_train_binary = vectorizer.fit_transform(X_train)
X_val_binary = vectorizer.transform(X_val)
X_test_binary = vectorizer.transform(X_test)
print(X_train_binary.shape[1])
# train logistic regression
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train_binary, y_train)
# evaluation and predict
y_pred_binary = logreg.predict(X_test_binary)

accuracy = logreg.score(X_test_binary, y_test)
f1_macro_binary = f1_score(y_test, y_pred_binary, average='macro')
f1_micro_binary = f1_score(y_test, y_pred_binary, average='micro')

print(f"Accuracy: {accuracy: .4f}")
print(f"F1 Macro Score (Binary):{f1_macro_binary: .4f}")
print(f"F1 Micro Score (Binary):{f1_micro_binary: .4f}")


61201
Accuracy:  0.9783
F1 Macro Score (Binary): 0.9402
F1 Micro Score (Binary): 0.9783


In [ ]:
# (b) frequency vector
# TODO
# CountVectorizer will count the frequency of vocabulary
Freq_Vectorizer = CountVectorizer()
# train vocabulary count
X_train_freq = Freq_Vectorizer.fit_transform(X_train)
X_val_freq = Freq_Vectorizer.transform(X_val)
X_test_freq = Freq_Vectorizer.transform(X_test)
# initalize logistic regression
logreg_freq = LogisticRegression(max_iter= 1000, random_state=42)
logreg_freq.fit(X_train_freq, y_train)

y_pred_freq = logreg_freq.predict(X_test_freq)

accuracy_freq = logreg_freq.score(X_test_freq, y_test)
f1_macro_freq = f1_score(y_test, y_pred_freq, average='macro')
f1_micro_freq = f1_score(y_test, y_pred_freq, average='micro')

print(f"Accuracy: {accuracy_freq: .4f}")
print(f"F1 Macro Score (Frequency):{f1_macro_freq: .4f}")
print(f"F1 Micro Score (Frequency):{f1_micro_freq: .4f}")

Accuracy:  0.9826
F1 Macro Score (Frequency): 0.9536
F1 Micro Score (Frequency): 0.9826


In [ ]:
# (c) tf-idf vector
# TODO
# initialize TF-IDF vectorizor
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
# initialize logistic regression
logreg_tfidf = LogisticRegression(max_iter=1000, random_state=42)
logreg_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = logreg_tfidf.predict(X_test_tfidf)

accuracy_tfidf = logreg_tfidf.score(X_test_tfidf, y_test)
f1_macro_tfidf = f1_score(y_test, y_pred_tfidf, average='macro')
f1_micro_tfidf = f1_score(y_test, y_pred_tfidf, average='micro')

print(f"Accuracy: {accuracy_tfidf: .4f}")
print(f"F1 Macro Score (TF-IDF):{f1_macro_tfidf: .4f}")
print(f"F1 Micro Score (TF-IDF):{f1_micro_tfidf: .4f}")

Accuracy:  0.9783
F1 Macro Score (TF-IDF): 0.9441
F1 Micro Score (TF-IDF): 0.9783


Part 2: Word2Vec (20 points):

Train a text classifier using the following document representation techniques using 100-dimensional
word vectors and report accuracy, macro-f1 score, and micro-f1 score on the test set. Compare and
analyze their performance.

(i) Using publicly available pre-trained Glove embeddings as word vectors, a document vector is
represented as an average of word vectors of its constituent words.

(ii) Train Word2Vec on NYT text data and use them as word vectors to compute document
vectors by averaging word vectors of its constituent words

In [ ]:
!pip install nltk gensim tqdm
import nltk
from tqdm import tqdm
from gensim.models import Word2Vec
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 29.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# TODO
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab') # Download the missing 'punkt_tab' resource
# convert to lowercase
def tokenize_and_lowercase(texts):
  tokenized_texts = [word_tokenize(text.lower()) for text in texts]
  return tokenized_texts
# split the train, val,test set
X_train_tokenized = tokenize_and_lowercase(X_train)
X_val_tokenized = tokenize_and_lowercase(X_val)
X_test_tokenized = tokenize_and_lowercase(X_test)
# load GLOVE word vectors
GLOVE_FILE = '/content/glove.6B.100d.txt'
EMBEDDING_DIM = 100
glove_embeddings = {}
# load Glove line by line
with open(GLOVE_FILE, 'r', encoding='utf-8') as f:
  for line in tqdm(f, desc = "Loading Glove Embeddings"):
    values = line.split()
    word = values[0]
    if len(values) >= EMBEDDING_DIM + 1:
      vector = np.asarray(values[1:], dtype='float32')
      glove_embeddings[word] = vector

def create_avg_word_vectors(tokenized_texts, embeddings, dim):
  doc_vectors = []
  # Create a zero vector for out of vocabulary words
  oov_vector = np.zeros(dim, dtype = 'float32')
  for tokens in tqdm(tokenized_texts, desc = "Averaging Vector"):
    # Find each Glove vector, use oov_vector if not found
    vectors = [embeddings.get(word,oov_vector) for word in tokens]
    if vectors:
      # Compute the average of words vector
      avg_vector = np.mean(np.stack(vectors), axis =0)
    else:
      avg_vector = oov_vector
    doc_vectors.append(avg_vector)
  return np.array(doc_vectors)
X_train_avg_vectors = create_avg_word_vectors(X_train_tokenized, glove_embeddings, EMBEDDING_DIM)
X_val_avg_vectors = create_avg_word_vectors(X_val_tokenized, glove_embeddings, EMBEDDING_DIM)
X_test_avg_vectors = create_avg_word_vectors(X_test_tokenized, glove_embeddings, EMBEDDING_DIM)
# initalize Logistic Regression
logreg_avg = LogisticRegression(max_iter=1000, random_state=42)
logreg_avg.fit(X_train_avg_vectors, y_train)
y_pred_avg = logreg_avg.predict(X_test_avg_vectors)

glove_accuracy = logreg_avg.score(X_test_avg_vectors, y_test)
glove_f1_macro = f1_score(y_test, y_pred_avg, average='macro')
glove_f1_micro = f1_score(y_test, y_pred_avg, average='micro')

print(f"Accuracy: {glove_accuracy: .4f}")
print(f"F1 Macro Score (Glove):{glove_f1_macro: .4f}")
print(f"F1 Micro Score (Glove):{glove_f1_micro: .4f}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
Loading Glove Embeddings: 400000it [00:09, 40974.17it/s]
Averaging Vector: 100%|██████████| 1152/1152 [00:00<00:00, 1553.07it/s]


Accuracy:  0.9731
F1 Macro Score (Glove): 0.9297
F1 Micro Score (Glove): 0.9731


In [1]:
# train the Word2Vec model
word2vec = Word2Vec(
    sentences = X_train_tokenized,
    vector_size = EMBEDDING_DIM,
    window = 5,
    min_count = 5,
    sg = 0,
    epochs = 10,
    workers = 4
)
self_trained_vectors = word2vec.wv
# Define the document vector averaging function
def create_avg_doc_vectors_w2v(tokenized_texts, w2v_model, dim):
  doc_vectors = []
  oov_vector = np.zeros(dim, dtype= 'float32')
  for tokens in tqdm(tokenized_texts, desc = "Averaging Vector"):
    vectors = []
    for word in tokens:
      # check if the word exists in the vocabulary learned by the w2v model
      if word in w2v_model.wv:
        vectors.append(w2v_model.wv[word])
      else:
        vectors.append(oov_vector)
    if vectors:
      # Calculate the average of all word vectors in the document
      avg_vector = np.mean(np.stack(vectors), axis = 0)
    else:
      avg_vector = oov_vector
    doc_vectors.append(avg_vector)
  return np.array(doc_vectors)
# Create Average Document Vectors for All Splits
X_train_w2v_vectors = create_avg_doc_vectors_w2v(X_train_tokenized, word2vec, EMBEDDING_DIM)
X_val_w2v_vectors = create_avg_doc_vectors_w2v(X_val_tokenized, word2vec, EMBEDDING_DIM)
X_test_w2v_vectors = create_avg_doc_vectors_w2v(X_test_tokenized, word2vec, EMBEDDING_DIM)
# Train the Classifier
logreg_w2v = LogisticRegression(max_iter=1000, random_state=42)
logreg_w2v.fit(X_train_w2v_vectors, y_train)
# Predict and Evaluate#
y_pred_w2v = logreg_w2v.predict(X_test_w2v_vectors)

w2v_accuracy = logreg_w2v.score(X_test_w2v_vectors, y_test)
w2v_f1_macro = f1_score(y_test, y_pred_w2v, average='macro')
w2v_f1_micro = f1_score(y_test, y_pred_w2v, average='micro')

print(f"Accuracy: {w2v_accuracy: .4f}")
print(f"F1 Macro Score (W2V):{w2v_f1_macro: .4f}")
print(f"F1 Micro Score (W2V):{w2v_f1_micro: .4f}")

NameError: name 'Word2Vec' is not defined

Part 3: BERT (20 points):

Fine-tune the BERT (bert-base-uncased) for text classification and report accuracy, macro f1-score,
and micro f1-score. If you are using PyTorch, hugging face transformers is highly recommended for
this task. While tokenizing, set the maximum length to 64 and fine-tune for 3 epochs.

Note, refer to this webpage for example (https://huggingface.co/google-bert/bert-base-uncased).
You will have to install required libraries by typing the following in your notebook.

In [ ]:
# Install & import required libraries
!pip install -q torch transformers scikit-learn tqdm
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, get_linear_schedule_with_warmup
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
# TODO
MAX_LEN = 64
NUM_EPOCHS = 3
BATCH_SIZE = 16
RANDOM_SEED = 42
# Prepare labels for the classifier head
unique_labels = y_train.unique()
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
num_labels = len(unique_labels)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class NYTDataset(torch.utils.data.Dataset):
  def __init__(self, texts, labels, tokenizer, max_len, label_to_id):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.label_to_id = label_to_id

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    # Use .iloc for safe positional indexing on pandas series
    text = str(self.texts.iloc[idx])
    label = self.labels.iloc[idx]
    # Tokenization and Encoding
    encoding = self.tokenizer(
    text,
    add_special_tokens=True,
    max_length=self.max_len,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
    )

    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'label': torch.tensor(self.label_to_id[label], dtype=torch.long)
    }
# Create Dataset instances
train_dataset = NYTDataset(X_train, y_train, tokenizer, MAX_LEN, label_to_id)
val_dataset = NYTDataset(X_val, y_val, tokenizer, MAX_LEN, label_to_id)
test_dataset = NYTDataset(X_test, y_test, tokenizer, MAX_LEN, label_to_id)

# Create DataLoaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Initalize BertForSequenceClassification with the correct number of output labels
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels,
    output_attentions=False,
    output_hidden_states=False
    )
# set device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Define the optimizer for BERT
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
# Training and Evaluation Functions
def train_epoch(model, data_loader, optimizer, device, scheduler):
  model = model.train()
  total_loss = 0.0
  for batch in tqdm(data_loader, desc="Training", leave = False):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["label"].to(device)

    optimizer.zero_grad()
    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels
    )
    loss = outputs.loss
    total_loss += loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
  avg_loss = total_loss / len(data_loader)
  return avg_loss

def eval_model(model, data_loader, device):
  model = model.eval()
  preds = []
  trues = []
  with torch.no_grad():
    for batch in tqdm(data_loader, desc="Evaluating", leave = False):
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch["attention_mask"].to(device)
      labels = batch["label"].to(device)
      outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask,
          labels=labels
      )
      logits = outputs.logits
      _, batch_preds = torch.max(logits, dim=1)
      preds.extend(batch_preds.cpu().numpy())
      trues.extend(labels.cpu().numpy())

  # Calculate Final Metrics Using accuracy_score
  accuracy = accuracy_score(trues, preds)
  f1_macro = f1_score(trues, preds, average='macro')
  f1_micro = f1_score(trues, preds, average='micro')
  return accuracy, f1_macro, f1_micro

# Loop to executes the required 3 epochs of training and evaluation
for epoch in range(NUM_EPOCHS):
  print(f"Epoch {epoch + 1}/{NUM_EPOCHS}")
  train_loss = train_epoch(model, train_loader, optimizer, device, scheduler)
  loss_fn = torch.nn.CrossEntropyLoss()
  val_accuracy, val_f1_macro, val_f1_micro = eval_model(model, val_loader, device)
  print(f"Train Loss: {train_loss:.4f}")
  print(f"Val   acc:  {val_accuracy:.4f}  |  F1_macro: {val_f1_macro:.4f}  |  F1_micro: {val_f1_micro:.4f}")
  test_acc, test_f1_macro, test_f1_micro = eval_model(model, test_loader, device)

  print(f"Accuracy: {test_acc:.4f}")
  print(f"F1 Macro Score: {test_f1_macro:.4f}")
  print(f"F1 Micro Score: {test_f1_micro:.4f}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Accuracy: 0.9766
F1 Macro Score: 0.9392
F1 Micro Score: 0.9766
Epoch 2/3


Train Loss: 0.0412
Val   acc:  0.9670  |  F1_macro: 0.9328  |  F1_micro: 0.9670


Accuracy: 0.9818
F1 Macro Score: 0.9554
F1 Micro Score: 0.9818
Epoch 3/3


Train Loss: 0.0119
Val   acc:  0.9696  |  F1_macro: 0.9384  |  F1_micro: 0.9696


Accuracy: 0.9800
F1 Macro Score: 0.9492
F1 Micro Score: 0.9800


Part 4: Summary of results / Reflection
## TODO Discuss your findings and results.

Part 1: Bag of Words

(a) Binary valued vector:

Accuracy:  0.9783  | F1 Macro Score (Binary): 0.9402 | F1 Micro Score (Binary): 0.9783

(b) Frequency Vector:

Accuracy:  0.9826 | F1 Macro Score (Frequency): 0.9536 | F1 Micro Score (Frequency): 0.9826

(c) TF-IDF Vector:

Accuracy:  0.9783 | F1 Macro Score (TF-IDF): 0.9441 | F1 Micro Score (TF-IDF): 0.9783

Part 2: Word2Vec

Accuracy:  0.9757 | F1 Macro Score (W2V): 0.9350 | F1 Micro Score (W2V): 0.9757

Part 3: BERT

Accuracy: 0.9800 | F1 Macro Score: 0.9492 | F1 Micro Score: 0.9800

From this experiment, I found that the simplest method, Bag of Words (Frequency Vector), yielded the best accuracy, even surpassing both the BERT and Word2Vec models. This outcome suggests that the data exhibits extremely high class distinction, allowing BoW to provide effective classification features. In contrast, Word2Vec performed poorly because it averaged the meaning of all constituent words, failing to provide accurate information regarding word importance. BERT did not achieve the highest performance because the implementation strictly required a maximum length truncation of 64 tokens, which severely restricted its ability to read the full context of the articles and utilize its superior contextual understanding capabilities. Finally, among the three BoW methods, Frequency vector proved the most effective, while the Binary value vector representation performed the worst due to its neglect of the repetitive importance of key vocabulary.
